In [5]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D,Activation, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers

In [6]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 250
l = 12
num_filter = 36 #added 24 more filters
compression = 0.5 
dropout_rate = 0.2

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [4]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False, kernel_regularizer = regularizers.l1() ,padding='same')(relu)
    #if dropout_rate>0:
      #Conv2D_BottleNeck = Dropout2D(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [5]:
# converted the last Dense Layer to a Fully Convolution N/w as use of Dense Layer was prohibited
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    temp = Conv2D(num_classes, kernel_size = (2,2))(AvgPooling)
    output = Activation('softmax')(temp)
    flat = Flatten()(output)
    
    return flat

In [6]:
# Dense Block
# removed the dropout
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        #if dropout_rate>0:
         # Conv2D_3_3 = Dropout2D(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [7]:
num_filter = 36
dropout_rate = 0.2
l= 12

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [8]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 36)   972         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 36)  144         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 36)   0           ['batch_normalization[0][0]']

In [9]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)

In [10]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(0.01, momentum = 0.7),
              metrics=['accuracy'])

In [11]:
from keras.callbacks import ModelCheckpoint, CSVLogger
ckpt = ModelCheckpoint('saved_model/model.hdf5')
csv = CSVLogger('saved_model/log.csv')

In [14]:
model.fit(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 10, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights('saved_model/10epochs.h5')

Epoch 1/10
390/390 [==============================] - 5049s 13s/step - loss: 5.5490 - accuracy: 0.4022 - val_loss: 2.6620 - val_accuracy: 0.2293
Epoch 2/10
390/390 [==============================] - 4852s 12s/step - loss: 1.9101 - accuracy: 0.4580 - val_loss: 2.1110 - val_accuracy: 0.3893
Epoch 3/10
390/390 [==============================] - 5140s 13s/step - loss: 1.6773 - accuracy: 0.5168 - val_loss: 1.7522 - val_accuracy: 0.4903
Epoch 4/10
390/390 [==============================] - 5576s 14s/step - loss: 1.5508 - accuracy: 0.5610 - val_loss: 1.8853 - val_accuracy: 0.4894
Epoch 5/10
390/390 [==============================] - 5482s 14s/step - loss: 1.4486 - accuracy: 0.5931 - val_loss: 3.5188 - val_accuracy: 0.2784
Epoch 6/10
390/390 [==============================] - 5136s 13s/step - loss: 1.3698 - accuracy: 0.6224 - val_loss: 1.4321 - val_accuracy: 0.6068
Epoch 7/10
390/390 [==============================] - 5018s 13s/step - loss: 1.3139 - accuracy: 0.6411 - val_loss: 1.4235 - val_ac

In [ ]:
# this will take more than 60 hours on my cpu so i haven't run this block
# also i got a descent accuracy of 71% on validation set only in 10 epoches.
model.fit(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights('saved_model/30epochs.h5')

In [ ]:
#you can run it for higher epoches by breaking it into parts and storing and restoring previous weights. 

#restoring the last model
from keras.models import load_model
model = load_model('saved_model/model.hdf5')
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights('saved_model/40epochs.h5')

In [3]:
from keras.models import load_model
model = load_model('saved_model/model.hdf5')

In [8]:
train_mse = model.evaluate(x_train, y_train, verbose=0)
test_mse = model.evaluate(x_test, y_test, verbose=0)

In [9]:
print('Train loss : %.3f %% , Test loss : %.3f %%' % (train_mse[0]*100, test_mse[0]*100))
print('Train accuracy : %.3f %% , Test accuracy : %.3f %%' % (train_mse[1]*100, test_mse[1]*100))

Train loss : 105.357 % , Test loss : 109.636 %
Train accuracy : 73.180 % , Test accuracy : 71.730 %
